In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import time
import argparse
import networkx as nx
import os
import pickle

import warnings
warnings.filterwarnings("ignore")

from st_graph_generation.detect_with_networkx import Node, Edge
from common_utils.stg_utils import load_video_params, load_dataset_params
from common_utils.process import graph_to_feature_vector, adj_to_normalized_tensor
from embedding_training.transformer_classifier import Net
from anomaly_generation.graph_corruption import Corruptor

VIDEOS_FOLDER = "../videos/"
GRAPH_FOLDER = "../graphs/"

TRAINING_GRAPH_FOLDER = "../training_graphs/"
VIDEO_PARAMS_FOLDER = "../video_parameters/"

TRAINING_PICKLE = 'train_dataset.pt'

In [ ]:
# to run only once to convert all the videos in VIDEOS_FOLDER into graphs (there are some errors in the videos_to_graphs function)
'''
with torch.no_grad():
    while True:
        try:
            videoname_to_graph_seq = detect_with_networkx.videos_to_graphs(source=VIDEOS_FOLDER, dest=GRAPH_FOLDER, save_graphs=True)
        except Exception as e:
            print(e)
            continue
        break
'''

In [ ]:
# code for reading the graphs saved with my method
'''
for videoname in os.listdir(GRAPH_FOLDER):
    if videoname.startswith('.'):
        continue
    videoname_folder = os.path.join(GRAPH_FOLDER, videoname)
    print(videoname_folder)
    # if in the list comprehension is to get only files for frames
    graph_paths = [os.path.join(videoname_folder, p) for p in os.listdir(videoname_folder) if p[0].isdigit()]
    graph_seq = [nx.read_gpickle(graph_path) for graph_path in graph_paths]

    # CORRUPTION
    video_params = load_video_params(videoname_folder)
    w, h = video_params["width"], video_params["height"]
    #TODO so far, is_stg must be true because only the distance is implemented on the edge as 'weight'
    corruptor = Corruptor(frame_width=w, frame_height=h, is_stg=True)

    corrupted_graph_seq = [corruptor.corrupt_graph(graph) for graph in graph_seq]
    
print(len(graph_seq))
print(len(corrupted_graph_seq))
'''

'''
for dataset_path in [file for file in os.listdir(GRAPH_FOLDER) if file.endswith(".pickle")]:
    with open(os.path.join(GRAPH_FOLDER, dataset_path), 'rb') as f:
        dataset = pickle.load(f)

        for videoname, seq_graph in dataset.items():
            print(videoname, len(seq_graph))
'''

# code to fix the keys in the pickle files (TODO you can delete as soon as the new process for generating data is consistent)
for dataset_path in [file for file in os.listdir(TRAINING_GRAPH_FOLDER)]:
    dataset_name = dataset_path.split("_")[0]
    print(dataset_name)
    if dataset_name!="Avenue":
        continue
    with open(os.path.join(TRAINING_GRAPH_FOLDER, dataset_path), 'rb') as f:
        dataset = pickle.load(f)
    #print(dataset.keys())
    
    #videoparams_path = os.path.join(VIDEO_PARAMS_FOLDER, dataset_name + "_video_params.pickle")
    #print(videoparams_path)
    #video2params = load_dataset_params(videoparams_path)

    new_dataset = dict()
    for key in dataset.keys():
        new_key = "".join(key.split(" "))
        print(f"{key} -> {new_key}")
        new_dataset[new_key] = dataset[key]
    
    with open("avenue_porcoddio.pickle", "wb") as f:
        pickle.dump(new_dataset, f, protocol=pickle.HIGHEST_PROTOCOL)
    


In [ ]:
'''DEBUGGING CORRUPTION PROCESS
for videoname in os.listdir(GRAPH_FOLDER):
    if videoname.startswith('.'):
        continue
    videoname_folder = os.path.join(GRAPH_FOLDER, videoname)
    video_params = load_video_params(videoname_folder)
    w, h = video_params["width"], video_params["height"]

    graph_paths = [os.path.join(videoname_folder, graph_path) for graph_path in os.listdir(videoname_folder)\
                if graph_path[0].isdigit()]
    
    graph_seq = [nx.read_gpickle(fpath) for fpath in graph_paths]

    corruptor = Corruptor(frame_width=w, frame_height=h, is_stg=True)
    for graph in graph_seq:
        s1 = set([n.id for n in graph.nodes])
        corr_graph = corruptor.corrupt_graph(graph)
        s2 = set([n.id for n in corr_graph.nodes])
        
        nx.adjacency_matrix(corr_graph)
'''

In [2]:
ANOMALY_LABEL = 1.
NORMAL_LABEL = 0.
NUM_NODE_FEATURES = 85 #TODO see if you can dynamically infer it
FRAMES_PER_VIDEOCLIP = 500 # about 15 seconds per clip
BATCH_SIZE = 1 #TODO batching problems, not working: I probably need to pad the features and adj vectors in __tensorize
#TODO or maybe using the collate_fn I can fix it


class GraphDataset(Dataset):
    def __init__(self, training_folder: str, videoparams_folder: str, frames_per_videoclip: int = None):
        self.frames_per_videoclip = frames_per_videoclip #to cut the video in videoclips
        self.__load_data_jasmine(training_folder, videoparams_folder)
    
    #TODO merge two methods for loading data (__load_data and __load_data_jasmine)
    def __load_data_jasmine(self, training_folder: str, videoparams_folder: str): 
        # TODO code for reading the graphs saved by Jasmine (make this function more readable?)

        self.tensored_videoclips = list()
        self.labels = list()

        for dataset_path in [file for file in os.listdir(training_folder)]:
            print(dataset_path)
            with open(os.path.join(training_folder, dataset_path), 'rb') as f:
                try:
                    dataset = pickle.load(f)
                except:
                    print(f"File {dataset_path} skipped")
                    continue
            
            dataset_name = dataset_path.split("_")[0]
            
            videoparams_path = os.path.join(videoparams_folder, dataset_name + "_video_params.pickle")
            video2params = load_dataset_params(videoparams_path)

            for videoname, graphs in dataset.items():
                video_width, video_height = video2params[videoname]["width"], video2params[videoname]["height"]

                # divide the video in videoclips if specified (many videoclips per videoname), otherwise one video per videoname
                if self.frames_per_videoclip and len(graphs) > self.frames_per_videoclip:
                    graph_seqs = [graphs[x:x+self.frames_per_videoclip] for x in range(0, len(graphs), self.frames_per_videoclip)]
                else:
                    graph_seqs = [graphs]
                
                # each videoclip in pytorch
                for graph_seq in graph_seqs:                
                    self.__process_and_corrupt_video_clip(graph_seq, video_width, video_height)

    def __load_data(self, training_folder: str):
        #TODO make the code of this function more readable

        self.tensored_videoclips = list()
        self.labels = list()

        for videoname in os.listdir(training_folder):
            if videoname.startswith('.') or videoname.endswith(".pickle"):
                continue
            videoname_folder = os.path.join(training_folder, videoname)
            print(videoname_folder)
            video_params = load_video_params(videoname_folder)
            video_width, video_height = video_params["width"], video_params["height"]
            
            # get all the files that contain graphs for videoname
            graph_paths = [os.path.join(videoname_folder, graph_path) for graph_path in os.listdir(videoname_folder)\
                if graph_path[0].isdigit()]
            graphs = [nx.read_gpickle(graph_path) for graph_path in graph_paths]
                            
            # divide the video in videoclips if specified (many videoclips per videoname), otherwise one video per videoname
            if self.frames_per_videoclip and len(graphs) > self.frames_per_videoclip:
                graph_seqs = [graphs[x:x+self.frames_per_videoclip] for x in range(0, len(graphs), self.frames_per_videoclip)]
            else:
                graph_seqs = [graphs]
                        
            # each videoclip in pytorch
            for graph_seq in graph_seqs:                
                self.__process_and_corrupt_video_clip(graph_seq, video_width, video_height)

    def __process_and_corrupt_video_clip(self, graph_seq, width, height):
        #TODO currently, a starting empty token is added to each sequence

        # NORMAL GRAPHS FOR CURRENT VIDEO
        features_and_adjs = [self.__get_empty_token()] + [self.__tensorize(graph) for graph in graph_seq]
        # padding
        while len(features_and_adjs) != self.frames_per_videoclip + 1: #TODO +1 because of the starting token
            features_and_adjs.append(self.__get_empty_token())

        self.tensored_videoclips.append(features_and_adjs)
        self.labels.append(torch.tensor(NORMAL_LABEL))
        
        # CORRUPTED GRAPHS FOR CURRENT VIDEO
        #TODO so far, is_stg must be true because only the distance is implemented on the edge as 'weight'
        corruptor = Corruptor(frame_width=width, frame_height=height, is_stg=True)
        corrupted_graph_seq = [corruptor.corrupt_graph(graph) for graph in graph_seq]

        corr_features_and_adjs = [self.__get_empty_token()] + [self.__tensorize(graph) for graph in corrupted_graph_seq]
        # padding
        while len(corr_features_and_adjs) != self.frames_per_videoclip + 1: #TODO +1 because of the starting token
            corr_features_and_adjs.append(self.__get_empty_token())

        self.tensored_videoclips.append(corr_features_and_adjs)
        self.labels.append(torch.tensor(ANOMALY_LABEL))
                
    def __tensorize(self, graph):
        if len(graph.nodes)==0:
            return self.__get_empty_token()
        features = graph_to_feature_vector(graph)
        adj = adj_to_normalized_tensor(nx.adjacency_matrix(graph))
        return features, adj
    
    def __get_empty_token(self):
        num_nodes = 20 #TODO this number in normal/abnormal graphs change everytime, see if defining a constant one is fine
        empty_features = torch.zeros(num_nodes, NUM_NODE_FEATURES)
        empty_adj = torch.zeros(num_nodes, num_nodes)
        return empty_features, empty_adj

    def __len__(self):
        return len(self.tensored_videoclips)

    def __getitem__(self, idx):
        # returning features, adjacency for each frame in the videoclip and the videoclip label
        return self.tensored_videoclips[idx], self.labels[idx]

train_set = GraphDataset(TRAINING_GRAPH_FOLDER, VIDEO_PARAMS_FOLDER, frames_per_videoclip=FRAMES_PER_VIDEOCLIP)
torch.save(train_set, TRAINING_PICKLE)
#train_set = GraphDataset(?????)
print(f"Number of clips: {len(train_set)}")

train_dataloader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=False)
#test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False)

Avenue_training.pickle
.DS_Store
File .DS_Store skipped
Adoc_training.pickle
Number of clips: 1202


In [6]:
def collate_fn(batch):
    return batch

train_set = torch.load(TRAINING_PICKLE)
train_dataloader = DataLoader(train_set, batch_size=1, shuffle=False) #, collate_fn=collate_fn) #TODO fix batching problem

for batch in iter(train_dataloader):
    seq, label = batch #len(seq) = FRAMES_PER_VIDEOCLIP + 1
    #print(label.shape)
    features, adj = seq[0]
    #print(features.shape)
    #print(adj.shape)
    #break

'''
torch.Size([2])
torch.Size([2, 20, 85])
torch.Size([2, 20, 20])
'''

'\ntorch.Size([2])\ntorch.Size([2, 20, 85])\ntorch.Size([2, 20, 20])\n'

In [ ]:
'''
features = graph_to_feature_vector(graph)
features.shape

adj = nx.adjacency_matrix(graph)
adj = adj_to_normalized_tensor(adj)
adj.shape
'''

In [4]:
''' TRAINING PART [WIP]'''
# source (TODO delete) https://n8henrie.com/2021/08/writing-a-transformer-classifier-in-pytorch/

from embedding_training.other_transformer_classifier import MyNet



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

epochs = 50
model = MyNet(
    features_size=NUM_NODE_FEATURES,
    embedding_size=8, #TODO decide a value for embedding vector (MUST BE EVEN and dividible by nhead) a low number is probably enough since most of the features are one-hot encoded ones
    max_length=FRAMES_PER_VIDEOCLIP + 1 #TODO because of the starting token
).to(device)

criterion = nn.BCELoss()
lr = 1e-4
optimizer = torch.optim.Adam(
    (p for p in model.parameters() if p.requires_grad), lr=lr
)
torch.manual_seed(0)

print("starting")
for epoch in range(epochs):
    epoch_loss = 0
    epoch_correct = 0
    epoch_count = 0
    for batch in iter(train_dataloader):
        seq_features_and_adjs, label = batch #len(seq) = FRAMES_PER_VIDEOCLIP + 1
        seq_features_and_adjs = [(features.to(device), adj.to(device)) for features, adj in seq_features_and_adjs]
        label = label.to(device)

        prediction = model.forward(seq_features_and_adjs)

        label = label.unsqueeze(1) # or prediction = prediction.squeeze(1)
        loss = criterion(prediction, label)

        label = label.squeeze(1)
        correct = prediction.argmax(axis=1) == label
        acc = correct.sum().item() / correct.size(0)

        epoch_correct += correct.sum().item()
        epoch_count += correct.size(0)

        epoch_loss += loss.item()

        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.5)

        optimizer.step()

    '''
    with torch.no_grad():
        test_epoch_loss = 0
        test_epoch_correct = 0
        test_epoch_count = 0

        for batch in iter(test_iter):
            predictions = model(batch.text.to(device))
            labels = batch.label.to(device) - 1
            test_loss = criterion(predictions, labels)

            correct = predictions.argmax(axis=1) == labels
            acc = correct.sum().item() / correct.size(0)

            test_epoch_correct += correct.sum().item()
            test_epoch_count += correct.size(0)
            test_epoch_loss += loss.item()
    '''

    print(f"{epoch=} - {epoch_loss=}, epoch accuracy: {epoch_correct / epoch_count}")
    #print(f"{test_epoch_loss=}, "test epoch accuracy: {test_epoch_correct / test_epoch_count}")

starting


KeyboardInterrupt: 

In [ ]:
"""
import math

import torch
import torch.nn as nn

import torchtext

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
batch_size = 30
max_length = 256

TEXT = torchtext.data.Field(
    lower=True, include_lengths=False, batch_first=True
)
LABEL = torchtext.data.Field(sequential=False)
train_txt, test_txt = torchtext.datasets.IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(
    train_txt,
    vectors=torchtext.vocab.GloVe(name="6B", dim=50, max_vectors=50_000),
    max_size=50_000,
)

LABEL.build_vocab(train_txt)

train_iter, test_iter = torchtext.data.BucketIterator.splits(
    (train_txt, test_txt),
    batch_size=batch_size,
)

for idx, batch in enumerate(iter(train_iter)):
    print(batch.text)
"""

# CORRUPTION
(see if it must be done during the process or it can be done as follows after all)

HOW CORRUPTUION CODE SHOULD BE CALLED

```
for videoclip:
    corruptor = Corruptor(..., ...)
    normal_sequence = list()
    corrupted_sequence = list()

    for frame in clip:
        graph = //graph initialization
        corrupted_graph = corruptor.corrupt_graph(graph)
        normal_sequence.append(graph)
        corrupted_sequence.append(corrupted_graph)
        // do we need object tracking etc.???
```